In [ ]:
import numpy as np
import os
from tensorflow.keras.models import model_from_json, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation, multiply, BatchNormalization
from tensorflow.keras import layers

from pystoi.stoi import stoi
import mir_eval #https://github.com/craffel/mir_eval
import librosa
import tensorflow as tf
from scipy.io import wavfile
from natsort import natsorted
from tensorflow.keras.models import load_model
from sklearn import preprocessing
import matplotlib.pyplot as plt
import pylab 
import soundfile as sf


In [ ]:
#define reconstruct function to reconstruct sound from framed signal.
def reconstruct(wave,angle):
    recon = np.sqrt(np.power(10, wave))
    recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=256, win_length=512, window='hann')
    return recon
def change_order(first):
    sec=np.copy(first)
    sec=np.copy(first)
    sec[0] = first[5]
    sec[1] = first[4]
    sec[2] = first[3]
    sec[5] = first[2]
    sec[4] = first[1]
    sec[3] = first[0]
    return sec

In [ ]:
# result_path = '/content/drive/My Drive/thesis/results'
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
batch_size = 32
timestep = 16
result_path = os.path.normpath(os.path.join(Data_path,'results'))
seed = 7
w=1
np.random.seed(seed)
result_path = os.path.normpath(os.path.join(Data_path,'results'))
seed = 7

In [ ]:
model_num = 106
loaded_model.load_weights(os.path.normpath(os.path.join(Data_path,'models','model_'+str(model_num)+'.h5')))

In [ ]:
def load_model_ckpt(model_num):
    h = [2048,2048,2048]
    w=7
    
    model = Sequential()
    model.add(Dense(h[0], input_dim = w*257))
    model.add(Activation('sigmoid'))
    model.add(Dense(h[1]))
    model.add(Activation('sigmoid'))
    model.add(Dense(h[2]))
    model.add(Activation('sigmoid'))
    model.add(Dense(257))
    model.load_weights(os.path.normpath(os.path.join(Data_path,'checkpoints',str(model_num),'weights.11.hdf5')))
    return model

In [ ]:
model_num_classify= 109
model_num_separate_male= 235
model_num_separate_female= 235
ckpt_num= '11'
import tensorflow as tf
import os
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
#either deine the model before or load it from a file here.
def load_model(model_num):
    Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
    json_file = open(os.path.normpath(os.path.join(Data_path,'models','model_'+str(model_num)+'.json')),'r')
    print('blah')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = tf.keras.models.model_from_json(loaded_model_json)
    loaded_model.load_weights(os.path.normpath(os.path.join(Data_path,'models','model_'+str(model_num)+'.h5')))
    print("Loaded model from disk")
    return loaded_model


model_classify = load_model(model_num_classify)
foldername_classify = 'results_'+str(model_num_classify)+'_test'
model_separate_male = load_model_ckpt(model_num_separate_male)
foldername_separate_male = 'results_'+str(model_num_separate_male)+'_test_male'
model_separate_female = load_model_ckpt(model_num_separate_female)
foldername_separate_female = 'results_'+str(model_num_separate_female)+'_test_female'
foldername_separate_both= 'results_'+str(model_num_separate_female)+'_'+str(model_num_separate_male)+'_test'
#change directories for ur own code.
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
write_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff/testdata_30h'
parent = 'results'
input_test_ftr = 'ftr_refrmd_test_w7_norm_withtrain'
input_test = 'test_log2_norm_withtrain'
inputs = os.listdir(os.path.normpath(os.path.join(write_path,input_test)))
inputs = natsorted(inputs)
#there are different directories since there are different SNRs.
if not os.path.exists(os.path.join(Data_path,parent,foldername_classify)):
    os.makedirs(os.path.join(Data_path,parent,foldername_classify))
if not os.path.exists(os.path.join(Data_path,parent,foldername_separate_male)):
    os.makedirs(os.path.join(Data_path,parent,foldername_separate_male))
if not os.path.exists(os.path.join(Data_path,parent,foldername_separate_female)):
    os.makedirs(os.path.join(Data_path,parent,foldername_separate_female))
if not os.path.exists(os.path.join(Data_path,parent,foldername_separate_both)):
    os.makedirs(os.path.join(Data_path,parent,foldername_separate_both))
predicts_snr = []
for snr_input in inputs:
    a = os.listdir(os.path.join(write_path,input_test,snr_input))
    if not os.path.exists(os.path.join(Data_path,parent,foldername_classify,snr_input)):
        os.mkdir(os.path.join(Data_path,parent,foldername_classify,snr_input))
    if not os.path.exists(os.path.join(Data_path,parent,foldername_separate_male,snr_input)):
        os.mkdir(os.path.join(Data_path,parent,foldername_separate_male,snr_input))
    if not os.path.exists(os.path.join(Data_path,parent,foldername_separate_female,snr_input)):
        os.mkdir(os.path.join(Data_path,parent,foldername_separate_female,snr_input))
    if not os.path.exists(os.path.join(Data_path,parent,foldername_separate_both,snr_input)):
        os.mkdir(os.path.join(Data_path,parent,foldername_separate_both,snr_input))
    predicts = []
    #predict the results base on the gender of the interferer.
    for filename_input in a:
        X_log=np.loadtxt(os.path.join(write_path,input_test,snr_input,filename_input),delimiter=',')
        prediction = model_classify.predict(X_log)
        for i in range(len(prediction)):
            if prediction[i][0]>0.5:
                prediction[i][0] = 1
            else:
                prediction[i][0] = 0
        lst = 0
        for i in range(len(prediction)):
            lst = lst+prediction[i][0]
        p_mean=lst/len(prediction)
        predicts.append(p_mean)
    predicts_snr.append(predicts)
np.savetxt(os.path.join(Data_path,parent,foldername_classify,'predict_gender_before_thresh.txt'),predicts_snr, fmt='%1.2f')
for i in range(len(predicts_snr)):
    for j in range(len(predicts_snr[i])):
        if predicts_snr[i][j]>0.5:
            predicts_snr[i][j] = 1
        else:
            predicts_snr[i][j] = 0
np.savetxt(os.path.join(Data_path,parent,foldername_classify,'predict_gender.txt'),predicts_snr, fmt='%1.0f')


inputs = os.listdir(os.path.normpath(os.path.join(write_path,input_test_ftr)))
inputs = natsorted(inputs)
phase = os.listdir(os.path.join(write_path,'test_phase2'))
phase = natsorted(phase)
count_snr = 0
for snr_input,snr_phase in zip(inputs,phase):
    a = os.listdir(os.path.join(write_path,input_test_ftr,snr_input))
    b = os.listdir(os.path.join(write_path,'test_phase2',snr_phase))
    count_file = 0
    for filename_input,filename_phase in zip(a,b):
        X_log=np.loadtxt(os.path.join(write_path,input_test_ftr,snr_input,filename_input),delimiter=',')
        X_phase=np.loadtxt(os.path.join(write_path,'test_phase2',snr_phase,filename_phase),delimiter=',')
        if predicts_snr[count_snr][count_file]==0:
            prediction = model_separate_male.predict(X_log)
            recon_out = reconstruct(prediction, X_phase)
            sf.write(os.path.normpath(os.path.join(Data_path,parent,foldername_separate_male,snr_input)+'\\'+filename_input.replace('.txt','')+'.wav'),recon_out,16000)
        elif predicts_snr[count_snr][count_file]==1:
            prediction = model_separate_female.predict(X_log)
            recon_out = reconstruct(prediction, X_phase)
            sf.write(os.path.normpath(os.path.join(Data_path,parent,foldername_separate_female,snr_input)+'\\'+filename_input.replace('.txt','')+'.wav'),recon_out,16000)
        sf.write(os.path.normpath(os.path.join(Data_path,parent,foldername_separate_both,snr_input)+'\\'+filename_input.replace('.txt','')+'.wav'),recon_out,16000)
        count_file +=1
    count_snr +=1
